In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor


In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import plot_confusion_matrix, recall_score,\
    accuracy_score, precision_score, f1_score

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline


from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier

In [4]:
X_train = pd.read_csv('../data/well_features.csv')
y_train = pd.read_csv('../data/well_labels.csv')
X_test = pd.read_csv('../data/well_test_labels.csv')

In [5]:
X_train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [6]:
dropped_list=['id','funder','wpt_name','num_private','basin','subvillage','region_code','district_code','lga','ward','recorded_by','scheme_name','extraction_type_group','extraction_type_class','management_group','payment_type','quality_group','quantity_group','source_type','source_class','waterpoint_type_group']

In [7]:
X_train = X_train.set_index(X_train['id'])
y_train = y_train.set_index(y_train['id'])
X_test = X_test.set_index(X_test['id'])
X_train = X_train.drop(dropped_list,axis=1)
X_test = X_test.drop(dropped_list,axis=1)
y_train = y_train.drop('id',axis=1)


In [8]:
y_train.head()

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional


In [9]:
X_test['date_recorded'] = X_test['date_recorded'].map(lambda x: x[:4]).astype('int64')
X_train['date_recorded'] = X_train['date_recorded'].map(lambda x: x[:4]).astype('int64')


In [11]:
X_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   amount_tsh         59400 non-null  float64
 1   date_recorded      59400 non-null  int64  
 2   gps_height         59400 non-null  int64  
 3   installer          55745 non-null  object 
 4   longitude          59400 non-null  float64
 5   latitude           59400 non-null  float64
 6   region             59400 non-null  object 
 7   population         59400 non-null  int64  
 8   public_meeting     56066 non-null  object 
 9   scheme_management  55523 non-null  object 
 10  permit             56344 non-null  object 
 11  construction_year  59400 non-null  int64  
 12  extraction_type    59400 non-null  object 
 13  management         59400 non-null  object 
 14  payment            59400 non-null  object 
 15  water_quality      59400 non-null  object 
 16  quantity          

In [ ]:
Gausian process
one hot encoding with lat and long

In [12]:
y_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status_group  59400 non-null  object
dtypes: object(1)
memory usage: 928.1+ KB


In [12]:
y_train['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [13]:
df_train = X_train.copy()
df_train = df_train.join(y_train, how='outer')

In [ ]:
#sns.pairplot(df_train)

In [14]:
df_train.isna().sum()

amount_tsh              0
date_recorded           0
gps_height              0
installer            3655
longitude               0
latitude                0
region                  0
population              0
public_meeting       3334
scheme_management    3877
permit               3056
construction_year       0
extraction_type         0
management              0
payment                 0
water_quality           0
quantity                0
source                  0
waterpoint_type         0
status_group            0
dtype: int64

In [19]:
#fig = px.scatter(df_train, x="construction_year", y="installer", color="status_group")
#fig.show()
#fig = px.bar(df, x="payment", y="status_group")
#fig.show()
#decision tree, knn, forest, bagging, GradientBoostingClassifier - sklearn.ensemble.HistGradientBoostingClassifier

In [28]:
fig = px.scatter(df_train, x="construction_year", y="amount_tsh", color="status_group")
fig.show()

In [18]:
def grab_numeric(df):
    return df.select_dtypes(include=['float', 'int'])

In [19]:
# The FunctionTransformer will turn my function
# into a transformer.

GrabNumeric = FunctionTransformer(grab_numeric)

In [20]:
pipe = Pipeline(steps=[('num', GrabNumeric),
                       ('ss', StandardScaler())])

In [32]:
pipe.fit(X_train)

Pipeline(steps=[('num',
                 FunctionTransformer(func=<function grab_numeric at 0x000002C1907E6550>)),
                ('ss', StandardScaler())])

In [33]:
pipe.transform(X_train)

array([[ 1.89566509,  0.13105173, -1.40879062],
       [-0.10597003,  0.09460999,  1.20793411],
       [-0.09762988,  0.51515818,  0.63975146],
       ...,
       [-0.10597003, -0.00918778, -1.03340379],
       [-0.10597003,  0.2716288 , -0.22828986],
       [-0.10597003,  0.61312481, -0.35350768]])

In [21]:
subpipe_num = Pipeline(steps=[('num_impute', SimpleImputer()),
                           ('ss', StandardScaler())])
subpipe_cat = Pipeline(steps=[('cat_impute', SimpleImputer(strategy='most_frequent')),
                             ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [60]:
#Pipeline Transformer
CT = ColumnTransformer(transformers=[('subpipe_num', subpipe_num, [0,1,2,4,5,7,11]),
                                         ('subpipe_cat', subpipe_cat, [3,6,8,9,10,12,13,14,15,16,17,18])],
                           remainder='passthrough')

In [57]:
#Models for Pipeline
clf1 = HistGradientBoostingClassifier(random_state = 2)
clf2 = RandomForestClassifier(random_state = 2)

In [61]:
#pipeline
pipe = Pipeline([('preprocessor', CT), ('classifier', clf1)])

In [64]:
#parameters for HistoGradient Booster
params1 = {}
params1['preprocessor']
params1['classifier__learning_rate'] = [.1, .5, 1]
params1['classifier__max_iter'] = [100, 500, 1000, 1500]
params1['classifier__l2_regularization'] = [0, 1, 2]
params1['classifier__max_depth'] = [5, 10, 15, 20]
params1['classifier'] = [clf1]

In [ ]:
#parameters for RandomForest
params2 = {}
params2['preprocessor']
params2['classifier__criterion'] = ['gini', 'entropy']
params2['classifier__n_estimators'] = [100, 200, 300, 500]
params2['classifier__min_samples_leaf'] = [1,5,10]
params2['classifier'] = [clf2]

In [ ]:
#parameters list
params = [params1, params2]

In [ ]:
#Search and results
grid = GridSearchCV(pipe, params)
grid.fit(X_train, y_train)
grid.best_params_

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgbc_model_pipe = Pipeline([('ct', CT), ('gbc', HistGradientBoostingClassifier(learning_rate=0.01, max_iter=500, max_depth=4, random_state=2))])

In [22]:
#Pipeline Transformer
CT = ColumnTransformer(transformers=[('subpipe_num', subpipe_num, [0,1,2,4,5,7,11]),
                                         ('subpipe_cat', subpipe_cat, [3,6,8,9,10,12,13,14,15,16,17,18])],
                           remainder='passthrough')

In [23]:
pipe1 = Pipeline(steps = [('preprocessor', CT), ('classifier', HistGradientBoostingClassifier(random_state = 2))])

In [24]:
parameters = {'classifier__learning_rate': [.1, 1], 'classifier__max_iter': [100], 'classifier__max_depth': [10], 'classifier__l2_regularization': [0]}

In [26]:
grid = GridSearchCV(estimator= pipe1, param_grid = parameters, verbose = 1)
grid.fit(X_train, y_train)
grid.best_params_

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A col

{'classifier__l2_regularization': 0,
 'classifier__learning_rate': 0.1,
 'classifier__max_depth': 10,
 'classifier__max_iter': 100}

In [28]:
grid.cv_results_['mean_test_score']

array([0.78659933, 0.72107744])

In [ ]:
#array([0.78659933, 0.72107744])

In [29]:
parameters2 = {'classifier__learning_rate': [.1], 'classifier__max_iter': [100], 'classifier__max_depth': [10,15], 'classifier__l2_regularization': [0]}

In [30]:
grid2 = GridSearchCV(estimator= pipe1, param_grid = parameters2, verbose = 1)
grid2.fit(X_train, y_train)
grid2.best_params_

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A col

{'classifier__l2_regularization': 0,
 'classifier__learning_rate': 0.1,
 'classifier__max_depth': 15,
 'classifier__max_iter': 100}

In [31]:
grid2.cv_results_['mean_test_score']

array([0.78659933, 0.7906734 ])

In [ ]:
#array([0.78659933, 0.7906734 ])

In [33]:
imb_pipe = ImPipeline(steps = [('preprocessor', CT), ('sm', SMOTE(sampling_strategy = 'auto', random_state = 2)), ('classifier', HistGradientBoostingClassifier(random_state = 2))])

In [34]:
parameters3 = {'classifier__learning_rate': [.1], 'classifier__max_iter': [100], 'classifier__max_depth': [15,20], 'classifier__l2_regularization': [0,2]}

In [36]:
grid3 = GridSearchCV(estimator= pipe1, param_grid = parameters3, verbose = 1)
grid3.fit(X_train, y_train)
grid3.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A col

{'classifier__l2_regularization': 0,
 'classifier__learning_rate': 0.1,
 'classifier__max_depth': 20,
 'classifier__max_iter': 100}

In [37]:
grid3.cv_results_['mean_test_score']

array([0.7906734 , 0.79089226, 0.78824916, 0.78875421])

In [ ]:
#array([0.7906734 , 0.79089226, 0.78824916, 0.78875421])

In [39]:
parameters4 = {'classifier__learning_rate': [.1], 'classifier__max_iter': [100, 200], 'classifier__max_depth': [20,25], 'classifier__l2_regularization': [0]}

In [40]:
grid4 = GridSearchCV(estimator= pipe1, param_grid = parameters4, verbose = 1)
grid4.fit(X_train, y_train)
grid4.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A col

{'classifier__l2_regularization': 0,
 'classifier__learning_rate': 0.1,
 'classifier__max_depth': 20,
 'classifier__max_iter': 200}

In [41]:
grid4.cv_results_['mean_test_score']

array([0.79089226, 0.79814815, 0.79040404, 0.79661616])

In [ ]:
#array([0.79089226, 0.79814815, 0.79040404, 0.79661616])

In [42]:
pipe2 = Pipeline(steps = [('preprocessor', CT), ('classifier', GradientBoostingClassifier(random_state = 2))])

In [45]:
parameters5 = {'classifier__learning_rate': [.1], 'classifier__max_depth': [20]}

In [46]:
grid5 = GridSearchCV(estimator= pipe2, param_grid = parameters5, verbose = 1)
grid5.fit(X_train, y_train)
grid5.best_params_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A col

{'classifier__learning_rate': 0.1, 'classifier__max_depth': 20}

In [47]:
grid5.cv_results_['mean_test_score']

array([0.80210438])

In [ ]:
#array([0.80210438])

In [51]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [52]:
pipe_gaus = Pipeline(steps = [('preprocessor', CT), ('classifier', GaussianProcessClassifier(random_state = 2))])

In [55]:
parameters6 = {}

In [56]:
grid6 = GridSearchCV(estimator= pipe_gaus, param_grid = parameters6, verbose = 1)
grid6.fit(X_train, y_train)
grid6.best_params_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\01aut\anaconda3\e

MemoryError: Unable to allocate 26.3 GiB for an array with shape (59400, 59400) and data type float64

In [58]:
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel, random_state=2).fit(X_train, y_train)

ValueError: could not convert string to float: 'Roman'

In [57]:

y_hat = grid5.predict(X_test)
y_hat

test = pd.read_csv('../data/well_test_labels.csv')
id_ = test['id']
results = pd.DataFrame({'id':id_,'status_group': y_hat})

results.to_csv('../data/results.csv')

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('subpipe_num',
                                                                         Pipeline(steps=[('num_impute',
                                                                                          SimpleImputer()),
                                                                                         ('ss',
                                                                                          StandardScaler())]),
                                                                         [0, 1,
                                                                          2, 4,
                                                                          5, 7,
                                                                          11]),
                                  

In [60]:
X_train_update = X_train.drop('installer',axis=1)
df_train_update = X_train_update.join(y_train, how='outer')


In [66]:
CT2 = ColumnTransformer(transformers=[('subpipe_num', subpipe_num, [0,1,2,3,4,7,10]),
                                         ('subpipe_cat', subpipe_cat, [5,7,8,9,11,12,13,14,15,16,17])],
                           remainder='passthrough')


In [67]:
pipe_gaus = Pipeline(steps = [('preprocessor', CT2), ('classifier', GaussianProcessClassifier(random_state = 2))])

In [68]:
parameters6 = {}

In [69]:
grid6 = GridSearchCV(estimator= pipe_gaus, param_grid = parameters6, verbose = 1)
grid6.fit(X_train_update, y_train)
grid6.best_params_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\01aut\anaconda3\envs\learn-env\lib\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\01aut\anaconda3\e

KeyboardInterrupt: 